In [1]:
import pickle
import numpy as np
import pandas as pd
import math
import time
from PIL import Image
# from patchify import patchify
# import albumentations as A
from IPython.display import SVG
#import graphviz
import matplotlib.pyplot as plt
%matplotlib inline
import os, re, sys, random, shutil #, cv2
from google.cloud import storage

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras import applications, optimizers
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications.resnet50 import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import model_to_dot, plot_model, image_dataset_from_directory
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger, LearningRateScheduler
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, ZeroPadding2D, Dropout

from sklearn.preprocessing import MinMaxScaler

2023-06-12 11:31:23.199019: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 11:31:24.142706: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-06-12 11:31:24.142831: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [2]:
# parameters
DATA_SIZE=1500
LOAD_CHUNK_SIZE=500
LAND_USE_ARRAY_SIZE=250
TRIAL_SIZE=200     # after running it once change to a larger number (up to 15000)
DATA_RUN = 'local_VM' #  select from these options:'gcp' local_VM' , 'local' 

# on Tim's GCP loading data from Bucket
BUCKET_NAME = "GlobalGreening"
images_dir = "zoomed_photos"
masks_dir = "ESA_worldcover"
training_output_dir = "training_outputs"

# on VM workbench in GCP when loading data locally from VM  - change to what is has to be 
VM_dataset_folder = "/home/jupyter/GlobalGreening"  # need to change this 
VM_images_dir = "zoomed_photos"
VM_masks_dir = 'ESA_worldcover'
VM_output_folder = "/home/jupyter/GlobalGreening/training_outputs" # need to change this 

### for local use - Alenka's computer
local_dataset_folder = "/Users/Alenka/code/Alastair908/GlobalGreening/raw_data"
local_images_dir = "images_trial_run"
local_masks_dir = "masks_trial_run"
local_output_folder = "/Users/Alenka/code/Alastair908/GlobalGreening/training_outputs"

In [3]:
# Define where to store model outputs

timestamp = time.strftime("%Y%m%d-%H%M%S")

if DATA_RUN == 'gcp':
    pass # not completed yet

elif DATA_RUN == 'local_VM':
    model_path = os.path.join(VM_output_folder, "models", f"{timestamp}_InceptionResNetV2-UNet.h5")
    results_path = os.path.join(VM_output_folder, "metrics", f"{timestamp}_model_training.csv")
    model_metrics_plot_path = os.path.join(VM_output_folder, "metrics", f"{timestamp}_model_metrics_plot.png")
    predictions_dir = os.path.join(VM_output_folder, f"{timestamp}_predictions")
    
elif DATA_RUN == 'local':
    model_path = os.path.join(local_output_folder, "models", f"{timestamp}_InceptionResNetV2-UNet.h5")
    results_path = os.path.join(local_output_folder, "metrics", f"{timestamp}_model_training.csv")
    model_metrics_plot_path = os.path.join(local_output_folder, "metrics", f"{timestamp}_model_metrics_plot.png")
    predictions_dir = os.path.join(local_output_folder,  f"{timestamp}_predictions")
    
print("--------")
print(f' model path is {model_path}')
print("--------")
print(f' results path is {results_path}')
print("--------")
print(f' model metrics path is {model_metrics_plot_path}')
print("--------")
print(f'predictions folder is {predictions_dir}')

--------
 model path is /home/jupyter/GlobalGreening/training_outputs/models/20230612-113206_InceptionResNetV2-UNet.h5
--------
 results path is /home/jupyter/GlobalGreening/training_outputs/metrics/20230612-113206_model_training.csv
--------
 model metrics path is /home/jupyter/GlobalGreening/training_outputs/metrics/20230612-113206_model_metrics_plot.png
--------
predictions folder is /home/jupyter/GlobalGreening/training_outputs/20230612-113206_predictions


In [ ]:
model.load_weights(model_path)

In [4]:
model_path = "/home/jupyter/GlobalGreening/training_outputs/models/20230611-082522_InceptionResNetV2-UNet.h5"

In [7]:
latest_model = keras.models.load_model(model_path, custom_objects={'CustomScaleLayer': CustomScaleLayer})

NameError: name 'CustomScaleLayer' is not defined